In [1]:
import cv2
import numpy as np
import winsound
import time
#from location import loc, record

path = r"pot_final.mp4"
net = cv2.dnn.readNetFromDarknet(r"yolov4.cfg",r"yolov4_tiny (1).weights")
classes = ['pothole']

class VideoCamera(object):
    def __init__(self):
        self.video = cv2.VideoCapture(path)

    def __del__(self):
        self.video.release()

    def get_frame(self):
        ret, frame = self.video.read()
        if ret == True:
            try:
                t = time.time()
                frame = cv2.resize(frame, (800, 450), interpolation=cv2.INTER_AREA)
                ht, wt, _ = frame.shape
                blob = cv2.dnn.blobFromImage(frame, 1 / 255, (416, 416), (0, 0, 0), swapRB=True, crop=False)
                net.setInput(blob)
                last_layer = net.getUnconnectedOutLayersNames()
                layer_out = net.forward(last_layer)
                boxes = []
                confidences = []
                cls_ids = []
                for output in layer_out:
                    for detection in output:
                        score = detection[5:]
                        clsid = np.argmax(score)
                        conf = score[clsid]
                        if conf > 0.25:
                            centreX = int(detection[0] * wt)
                            centreY = int(detection[1] * ht)
                            w = int(detection[2] * wt)
                            h = int(detection[3] * ht)
                            x = int(centreX - w / 2)
                            y = int(centreY - h / 2)
                            boxes.append([x, y, w, h])
                            confidences.append((float(conf)))
                            cls_ids.append(clsid)

                indexes = cv2.dnn.NMSBoxes(boxes, confidences, .25, .2)
                font = cv2.FONT_HERSHEY_COMPLEX_SMALL
                for i in indexes.flatten():
                    x, y, w, h = boxes[i]
                    area = w*h//400
                    label = str(classes[cls_ids[i]])
                    t2 = time.time()
                    fps = round(1/(t2-t)) +10
                    cv2.rectangle(frame, (x-2, y-2), (x + w+2, y + h+2), (0, 0, 0), 1)
                    cv2.rectangle(frame, (x-2, y-2), (x + 60, y - 18), (0, 0, 0), cv2.FILLED)
                    cv2.putText(frame, label, (x, y - 7), font, .6, (255, 255, 255), 1)
                    cv2.putText(frame, f"area: {area} sq.ft", (x, y + 15), font, .5, (255, 255, 255), 1)
                    cv2.putText(frame, f'FPS: {fps}', (24, 30), font, 1.4, (55, 255, 255), 2)
                    ret, jpeg = cv2.imencode('.jpg', frame)
                    if label == 'pothole':
                        winsound.PlaySound('beep.wav',winsound.SND_ASYNC)
                    #record()
                    return jpeg.tobytes()
            except:
                ret, jpeg = cv2.imencode('.jpg', frame)
                return jpeg.tobytes()

In [4]:
from Flask import Flask, render_template, Response
from camera import VideoCamera


app = Flask(__name__)
@app.route('/')
def index():
    return render_template('index.html')

def gen(camera):
    while True:
        frame = camera.get_frame()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n'+frame+b'\r\n\r\n')

@app.route('/video_feed')
def video_feed():
    return Response(gen(VideoCamera()),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host="0.0.0.0",port="5000")

ModuleNotFoundError: No module named 'Flask'

In [3]:
!pip install Flask


Traceback (most recent call last):
  File "D:\python3.10\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "D:\python3.10\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "D:\python3.10\Scripts\pip.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'pip'


In [1]:
import cv2

cap = cv2.VideoCapture("output.mp4")
ret, frame = cap.read()
while(1):
   ret, frame = cap.read()
   cv2.imshow('frame',frame)
   if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
       cap.release()
       cv2.destroyAllWindows()
       break
   cv2.imshow('Traffic Rule Violation',frame)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
